In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import lxml

# Wikipedia URL for Dow Jones components
url = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'

# Use headers to mimic a browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}

html_data=requests.get(url, headers=headers).text
jsoup=BeautifulSoup(html_data,'lxml')
jsoup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Dow Jones Industrial Average - Wikipedia\n  </title>\n  <script>\n   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature

In [2]:
tickers=jsoup.find_all('tr')
tickers[1].td.text
tickers

[<tr class="image"><td class="infobox-image" colspan="2"><span typeof="mw:File"><a class="mw-file-description" href="/wiki/File:DJIA_historical_graph_(log).svg"><img alt="A historical graph. The Dow rises periodically through the decades with corrections along the way, from its record low of under 35 in the late 1890s to a high of around 36,000 in 2022." class="mw-file-element" data-file-height="400" data-file-width="800" decoding="async" height="150" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/DJIA_historical_graph_%28log%29.svg/330px-DJIA_historical_graph_%28log%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/DJIA_historical_graph_%28log%29.svg/500px-DJIA_historical_graph_%28log%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/cd/DJIA_historical_graph_%28log%29.svg/600px-DJIA_historical_graph_%28log%29.svg.png 2x" width="300"/></a></span><div class="infobox-caption">Historical logarithmic graph of the DJIA from 1896 to 2018</div><

In [3]:
tick_list=[]

for t in tickers:
    td_elements = t.find_all('td')

    if len(td_elements) >= 2:
        ticker_symbol = td_elements[1].text.strip()
        tick_list.append(ticker_symbol)
dow_tickers=tick_list[:30]
dow_tickers=dow_tickers[1:]
len(dow_tickers)

29

In [4]:
from datetime import date
with open ('fredapi_text.txt', 'r') as fred_file: #name file fredapi_text.txt with personal fredapi key to retreive risk free rate
    fred_apikey=fred_file.read().strip()

#Retrieve risk-free rate from FRED
from fredapi import Fred
fred=Fred(api_key=fred_apikey)
tenyr_yield_hist = fred.get_series("GS10") / 100
#Take the first ten year yield in the sample period from FRED
start_date=date(2010,1,1)
rf=float(tenyr_yield_hist[tenyr_yield_hist.index>=pd.to_datetime(start_date)].iloc[0])
rf

0.0373

In [5]:
import yfinance as yf

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)
dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Close']

[*********************100%***********************]  29 of 29 completed


In [6]:
#Minimum variance portfolio in a simulation
optport = []

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)

    dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']

    monthlyprc=dailyprc.resample('M').last()
    monthlyrets=monthlyprc.pct_change().dropna()

    # Average returns per year
    mu_vec=np.array(monthlyrets.mean()*12)

    #Covariance matrix
    cov_mat=np.array(monthlyrets.cov()*12)

    num_simul=10000
    N=len(dow_tickers)
    port_expret=np.zeros(num_simul) #vector to capture the expret of each simulation
    port_var=np.zeros(num_simul) #vector to capture the variance of each simulation
    np.random.seed(100) #set seed so results are comparable
    weights=np.zeros((num_simul, N)) #making it a matrix where each row is a simulation

    for i in range(0,num_simul):
        #i in the for loop references the simulation number

        temp=np.random.rand(N) # choose N random numbers from uniform distribution
        weights[i]=temp/temp.sum() #forcing the sum of weights to =1

        #dimensions of each variable are:

        #temp is a VECTOR (array) of N random numbers
        #weights is a MATRIX of size num_simul X N
        #each row of "weights" are the N random numbers normalized to sum to 1
        #each row of "weights" is a different simulation

        #Expected Return formula (wT * R)
        port_expret[i] = np.dot(weights[i].transpose(), mu_vec)

        #Portfolio Variance formula (wT * COVmat * w)
        port_var[i] = np.dot(weights[i].transpose(), np.dot(cov_mat, weights[i]))

    port_sd=np.sqrt(port_var)

    #Minimum variance portfolio in a simulation
    port_var.argmin()
    minvar_sim=weights[port_var.argmin()]

    optweights=pd.DataFrame(minvar_sim, index=dow_tickers, columns=[f'{y}-{y + 2-2000}']) #create a new dataframe
    optport.append(optweights)

optport_df = pd.concat(optport, axis=1)
optport_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2532852382.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2532852382.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2532852382.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2532852382.py:10: FutureWarn

,2010-12,2011-13,2012-14,2013-15,2014-16,2015-17,2016-18,2017-19,2018-20,2019-21,2020-22,2021-23
MMM,0.023120,0.069130,0.018240,0.037237,0.004517,0.004517,0.031215,0.031215,0.009766,0.009766,0.009766,0.027367
AXP,0.044158,0.064429,0.065381,0.024453,0.001778,0.001778,0.012981,0.012981,0.015035,0.015035,0.015035,0.058860
AMGN,0.049474,0.000024,0.004903,0.008302,0.014746,0.014746,0.002221,0.002221,0.077344,0.077344,0.077344,0.021515
AMZN,0.001315,0.008881,0.010962,0.029837,0.013620,0.013620,0.044840,0.044840,0.004396,0.004396,0.004396,0.025807
AAPL,0.034252,0.020872,0.007750,0.016731,0.061572,0.061572,0.019308,0.019308,0.005139,0.005139,0.005139,0.004419
BA,0.013806,0.014858,0.017815,0.011278,0.036288,0.036288,0.002186,0.002186,0.089182,0.089182,0.089182,0.014437
CAT,0.011998,0.008574,0.010924,0.033899,0.017510,0.017510,0.023833,0.023833,0.008873,0.008873,0.008873,0.037197
CVX,0.010926,0.014433,0.044753,0.064705,0.024091,0.024091,0.063016,0.063016,0.003669,0.003669,0.003669,0.034027
CSCO,0.039019,0.062948,0.046045,0.045832,0.023682,0.023682,0.024508,0.024508,0.014627,0.014627,0.014627,0.051193
KO,0.015328,0.043187,0.059930,0.012445,0.020735,0.020735,0.049644,0.049644,0.019832,0.019832,0.019832,0.016417


In [7]:
#max Sharpe portfolio in a simulation
optport = []

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)

    dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']

    monthlyprc=dailyprc.resample('M').last()
    monthlyrets=monthlyprc.pct_change().dropna()

    # Average returns per year
    mu_vec=np.array(monthlyrets.mean()*12)

    #Covariance matrix
    cov_mat=np.array(monthlyrets.cov()*12)

    num_simul=10000
    N=len(dow_tickers)
    port_expret=np.zeros(num_simul) #vector to capture the expret of each simulation
    port_var=np.zeros(num_simul) #vector to capture the variance of each simulation
    np.random.seed(100) #set seed so results are comparable
    weights=np.zeros((num_simul, N)) #making it a matrix where each row is a simulation

    for i in range(0,num_simul):
        #i in the for loop references the simulation number

        temp=np.random.rand(N) # choose N random numbers from uniform distribution
        weights[i]=temp/temp.sum() #forcing the sum of weights to =1

        #You should be very clear what the dimensions of each variable are

        #temp is a VECTOR (array) of N random numbers
        #weights is a MATRIX of size num_simul X N
        #each row of "weights" are the N random numbers normalized to sum to 1
        #each row of "weights" is a different simulation

        #Expected Return formula (wT * R)
        port_expret[i] = np.dot(weights[i].transpose(), mu_vec)

        #Portfolio Variance formula (wT * COVmat * w)
        port_var[i] = np.dot(weights[i].transpose(), np.dot(cov_mat, weights[i]))

    port_sd=np.sqrt(port_var)

    tenyr_yield_hist=fred.get_series("GS10")/100
    #Take the first ten year yield in the sample period from FRED
    rf=tenyr_yield_hist[tenyr_yield_hist.index>=pd.to_datetime(start_date)][0]

    sharpes=(port_expret-rf)/port_sd

    #max Sharpe portfolio in a simulation
    sharpes.argmax()

    maxSR_sim=weights[sharpes.argmax()]

    optweights=pd.DataFrame(maxSR_sim, index=dow_tickers, columns=[f'{y}-{y + 2-2000}']) #create a new dataframe
    optport.append(optweights)

optport_df = pd.concat(optport, axis=1)
optport_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2128151617.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2128151617.py:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rf=tenyr_yield_hist[tenyr_yield_hist.index>=pd.to_datetime(start_date)][0]
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2128151617.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\212

,2010-12,2011-13,2012-14,2013-15,2014-16,2015-17,2016-18,2017-19,2018-20,2019-21,2020-22,2021-23
MMM,0.057832,0.033757,0.033757,0.032458,0.027543,0.027543,0.021504,0.063463,0.063028,0.090886,0.068312,0.062927
AXP,0.052428,0.039852,0.039852,0.014728,0.016682,0.016682,0.010273,0.072073,0.028206,0.004778,0.006454,0.028168
AMGN,0.023932,0.025073,0.025073,0.066121,0.016609,0.016609,0.070625,0.001902,0.103207,0.029094,0.023087,0.012675
AMZN,0.066130,0.033689,0.033689,0.039514,0.009567,0.009567,0.055547,0.018046,0.000958,0.044068,0.020374,0.056931
AAPL,0.003883,0.023845,0.023845,0.022930,0.027414,0.027414,0.084720,0.073406,0.017625,0.000821,0.002141,0.005826
BA,0.015085,0.002067,0.002067,0.002112,0.017461,0.017461,0.019076,0.004602,0.005646,0.040139,0.037917,0.002813
CAT,0.056647,0.028513,0.028513,0.022571,0.011178,0.011178,0.015025,0.060433,0.002464,0.019403,0.001927,0.004067
CVX,0.003871,0.015853,0.015853,0.018513,0.055848,0.055848,0.065852,0.042230,0.005408,0.017566,0.033016,0.013942
CSCO,0.044812,0.014693,0.014693,0.000931,0.040824,0.040824,0.006385,0.006487,0.028652,0.006221,0.002270,0.068429
KO,0.010247,0.054178,0.054178,0.050759,0.001493,0.001493,0.026428,0.005170,0.038731,0.001487,0.006704,0.014024


In [8]:
#max Sharpe portfolio analytically
optport = []

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)

    dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']

    monthlyprc=dailyprc.resample('M').last()
    monthlyrets=monthlyprc.pct_change().dropna()

    # Average returns per year
    mu_vec=np.array(monthlyrets.mean()*12)

    #Covariance matrix
    cov_mat=np.array(monthlyrets.cov()*12)

    tenyr_yield_hist=fred.get_series("GS10")/100
    #Take the first ten year yield in the sample period from FRED
    rf=tenyr_yield_hist[tenyr_yield_hist.index>=pd.to_datetime(start_date)][0]

    #max Sharpe portfolio analytically
    Sinv=np.linalg.inv(cov_mat)
    numer=np.dot(Sinv,mu_vec-rf)
    denom=np.dot(np.ones(N), np.dot(Sinv,mu_vec-rf))
    maxSR_math=numer/denom

    optweights=pd.DataFrame(maxSR_math, index=dow_tickers, columns=[f'{y}-{y + 2-2000}']) #create a new dataframe
    optport.append(optweights)

optport_df = pd.concat(optport, axis=1)
optport_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\4177863906.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\4177863906.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rf=tenyr_yield_hist[tenyr_yield_hist.index>=pd.to_datetime(start_date)][0]
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\4177863906.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\417

,2010-12,2011-13,2012-14,2013-15,2014-16,2015-17,2016-18,2017-19,2018-20,2019-21,2020-22,2021-23
MMM,-0.115776,0.067739,0.180005,0.526132,-0.337887,-0.407937,0.301376,-0.142408,-0.500359,-0.084963,0.544775,0.436661
AXP,2.084460,0.742593,-0.569187,0.110415,-0.842535,-1.046499,-0.309512,-0.313105,-1.231667,0.600294,-2.162775,0.800525
AMGN,-0.678438,0.545094,-0.040793,-0.075609,-0.216510,-0.381583,-0.179195,-0.136116,0.783097,-0.823290,-1.181849,0.884770
AMZN,-0.239083,0.235414,-3.369999,0.948842,-2.347914,-1.018289,-0.792764,-2.261291,-1.840555,0.813666,1.991139,0.761129
AAPL,-0.410903,-0.501703,-1.500615,0.155891,-0.587860,0.398579,0.070562,0.881641,0.697069,-0.476178,0.755853,-0.317597
BA,0.348766,-0.032629,-1.073878,0.114897,0.448376,0.063059,0.271129,-0.777586,0.440876,-0.349020,-0.839299,1.173922
CAT,0.645627,-0.375176,0.512925,0.240150,-1.070205,-0.643301,0.276436,-0.494634,-0.616752,-0.258834,-3.723408,-0.043344
CVX,0.246610,-0.346556,0.715636,0.380284,1.980221,0.794573,0.719849,0.779129,-0.398696,-1.014326,-1.998409,-0.298144
CSCO,0.912748,0.223396,-0.227660,0.968723,2.492053,0.069371,0.444315,1.448526,0.380503,-0.637081,-1.473619,-0.583400
KO,0.433704,0.174472,2.357213,-0.868894,-0.765897,-0.324745,0.021162,0.037789,-0.612011,-0.828510,-0.321577,-1.589159


In [9]:
#max Sharpe portfolio analytically
#with the constraint that each weight must be positive and no weight can be bigger than 10%
from scipy.optimize import minimize
optport = []

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)

    dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Close']

    monthlyprc=dailyprc.resample('M').last()
    monthlyrets=monthlyprc.pct_change().dropna()

    # Average returns per year
    mu_vec=np.array(monthlyrets.mean()*12)

    #Covariance matrix
    cov_mat=np.array(monthlyrets.cov()*12)


    def portsd_func(weights,covmat):
        return np.sqrt(np.dot(weights.transpose(), np.dot(covmat, weights)))

    def neg_sharpe(weights, muret, covmat, rf):
        #weights is a vector of length N
        #muret is a vector of mean returns of length N
        #covmat is the var-cov matrix
        #rf is risk free rate
        
        return -1*((np.dot(weights.transpose(), mu_vec)-rf)/portsd_func(weights, covmat))

    constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    #'type': 'eq' means constraining function has to equal zero
    #'fun': is the function defining the constraint, which is that the weights must sum to 1
    #writing a function to make sum(weights)-1 equal to 0
    #a constraint is an exact equation

    bounds = [(0, .1) for _ in range(N)]
    #limiting what the weights can be
    #creating a list called bounds which is [(0,0.4), (0,0.4)...etc]
    #for all the stocks
    #bounds are less than or greater than

    initial_weights = np.ones(N)/N 


    optimized_results = minimize(neg_sharpe, initial_weights, args=(mu_vec, cov_mat, rf), method='SLSQP', constraints=constraints, bounds=bounds)

    constr_opt=optimized_results.x

    optweights=pd.DataFrame(constr_opt.round(6), index=dow_tickers, columns=[f'{y}-{y + 2-2000}']) #create a new dataframe
    optport.append(optweights)

optport_df = pd.concat(optport, axis=1)
optport_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2564517605.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2564517605.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2564517605.py:12: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2564517605.py:12: FutureWarn

,2010-12,2011-13,2012-14,2013-15,2014-16,2015-17,2016-18,2017-19,2018-20,2019-21,2020-22,2021-23
MMM,0.100000,0.053173,0.002691,0.048039,0.060062,0.000000,0.000000,0.085959,0.100000,0.100000,0.100000,0.0
AXP,0.100000,0.100000,0.100000,0.018949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
AMGN,0.000000,0.100000,0.000000,0.060183,0.100000,0.100000,0.000000,0.000000,0.100000,0.048288,0.000000,0.0
AMZN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.048380,0.000000,0.000000,0.000000,0.000000,0.1
AAPL,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.100000,0.044083,0.000000,0.000000,0.000000,0.0
BA,0.000000,0.000000,0.000000,0.000000,0.000000,0.051575,0.039289,0.000000,0.000000,0.065031,0.100000,0.0
CAT,0.100000,0.000000,0.028281,0.039382,0.000000,0.000000,0.100000,0.055466,0.027207,0.000000,0.000000,0.0
CVX,0.000000,0.000000,0.000059,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0
CSCO,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.1
KO,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013202,0.000000,0.000000,0.0


In [10]:
#GMV portfolio analytically
optport = []

for y in range(2010,2022):
    start_date=date(y,1,1)
    end_date=date(y+2,12,31)

    dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']

    monthlyprc=dailyprc.resample('M').last()
    monthlyrets=monthlyprc.pct_change().dropna()

    # Average returns per year
    mu_vec=np.array(monthlyrets.mean()*12)

    #Covariance matrix
    cov_mat=np.array(monthlyrets.cov()*12)

    #Construct matrix A
    np.set_printoptions(suppress=True, precision=4)
    TwoS=2*cov_mat

    Leftside=np.insert(TwoS,N, np.ones(N), axis=0)

    Lastcol=np.ones(N)
    Lastcol=np.append(Lastcol,0)
    Amat=np.insert(Leftside,N, Lastcol, axis=1)
    b_vec=np.zeros(N)
    b_vec=np.append(b_vec,1)
    Amat_inv=np.linalg.inv(Amat)
    solution=np.dot(Amat_inv,b_vec)
    gmv_weights=solution[0:N]


    optweights=pd.DataFrame(gmv_weights, index=dow_tickers, columns=[f'{y}-{y + 2-2000}']) #create a new dataframe
    optport.append(optweights)

optport_df = pd.concat(optport, axis=1)
optport_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1649127876.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1649127876.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1649127876.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1649127876.py:10: FutureWarn

,2010-12,2011-13,2012-14,2013-15,2014-16,2015-17,2016-18,2017-19,2018-20,2019-21,2020-22,2021-23
MMM,0.065946,0.195172,0.004961,0.230715,0.096274,-0.075022,0.112747,0.128347,-0.291561,-0.376361,-0.582026,0.248979
AXP,0.119613,-0.035348,-0.188058,-0.019520,-0.391666,-0.257344,0.093314,0.019285,0.332864,0.177841,-0.889993,0.067399
AMGN,-0.253802,0.047460,-0.083706,0.012736,0.240228,-0.127222,-0.246224,-0.296997,0.030502,0.023237,0.016378,0.589215
AMZN,0.067855,0.047014,-0.114551,0.388796,0.035512,0.262367,-0.775302,-0.674077,-1.088380,-0.226126,0.240204,0.150609
AAPL,-0.012093,0.022423,-0.025055,-0.136221,-0.222318,-0.188393,-0.265433,0.172494,-0.005827,-0.004578,0.419384,-0.269608
BA,-0.157070,-0.062576,0.145460,0.140383,0.035456,0.023562,0.293444,-0.170399,0.182685,-0.183272,-0.228672,0.222252
CAT,0.045596,-0.228602,-0.011841,0.177939,-0.025172,0.018155,0.291672,0.303023,0.272853,-0.011542,-0.920242,0.169745
CVX,-0.037291,-0.029850,-0.094300,0.272463,-0.035358,-0.358797,0.447169,0.472420,0.480858,-0.067448,-0.210150,0.091915
CSCO,0.114723,-0.011682,0.076326,0.369208,0.307521,0.118919,0.381107,0.510326,0.038927,-0.288856,-0.299175,-0.038800
KO,-0.224821,-0.340001,-0.594214,-0.561052,-0.425407,0.058955,0.340345,0.113036,-0.324304,-0.295169,0.627150,-0.412609


In [11]:
""""The weights of each ticker in the five portfolios all change substantially between each window.
There are few instance where tickers maintain dominant weight in the portfolio. Because of the inconsistency in
appropriate weights from year to year, it would likely be difficult to acheive the same risk-return outcome
use the same weights in a subsequent period."""


'"The weights of each ticker in the five portfolios all change substantially between each window.\nThere are few instance where tickers maintain dominant weight in the portfolio. Because of the inconsistency in\nappropriate weights from year to year, it would likely be difficult to acheive the same risk-return outcome\nuse the same weights in a subsequent period.'

In [12]:
start_date=date(2010,1,1)
end_date=date(2023,12,31)
dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']
monthlyprc=dailyprc.resample('M').last()
monthlyrets=monthlyprc.pct_change()

window_rets_df = pd.DataFrame(columns=["weights window",'investment period', 'predicted return', "cumulative_return"])

for i in range(0, len(optport_df.columns)-1):
    startyr_weights = int(optport_df.columns[i][:4])  # ex: '2010-12' -> '2010'

    return_period_start = startyr_weights + 3
    return_period_end = return_period_start + 2

    return_start = f"{return_period_start}"
    return_end = f"{return_period_end}"

    predicted_portfolio_weights = optport_df.iloc[:, i+1]
    predrets = (predicted_portfolio_weights * monthlyrets.loc[return_start:return_end]).sum(axis=1)  # Predicted returns for this period
    predcumreturn = np.prod(1 + predrets)

    portfolio_weights = optport_df.iloc[:, i]
    returns = (portfolio_weights * monthlyrets.loc[return_start:return_end]).sum(axis=1)

    cumreturn = np.prod(1 + returns)

    new_row = pd.DataFrame({
        "weights window": [f"{startyr_weights}-{startyr_weights + 2}"], 'investment period': [f"{return_period_start}-{return_period_end}"],
        'predicted return':[predcumreturn],"cumulative_return": [cumreturn]
    })

    window_rets_df = pd.concat([window_rets_df, new_row], ignore_index=True)

window_rets_df

[*********************100%***********************]  29 of 29 completed
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1996470793.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthlyprc=dailyprc.resample('M').last()
C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\1996470793.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  window_rets_df = pd.concat([window_rets_df, new_row], ignore_index=True)


,weights window,investment period,predicted return,cumulative_return
0,2010-2012,2013-2015,2.743565,0.919696
1,2011-2013,2014-2016,2.787366,1.844630
2,2012-2014,2015-2017,8.451341,2.863279
3,2013-2015,2016-2018,2.396614,4.907343
4,2014-2016,2017-2019,1.844058,1.344407
5,2015-2017,2018-2020,0.256407,2.217736
6,2016-2018,2019-2021,1.575687,0.342034
7,2017-2019,2020-2022,0.650625,2.733700
8,2018-2020,2021-2023,1.462925,1.134083
9,2019-2021,2022-2024,0.226511,1.117120


In [13]:
#Conduct a VaR analysis on an equally weighted portfolio of the 30 stocks for the entire sample period
#using the three methods we discussed in class (Parametric, Historical, Monte Carlo).
#Use days=30 and an initial portfolio value of $100.

#VaR with stock daily data

N=len(dow_tickers)
start_date=date(2010,1,1)
end_date=date(2023,12,31)

dailyprice=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)['Adj Close']

days=30
portfolio_value=100

#Define log returns
lnret=np.log(dailyprc/dailyprc.shift(1))  #shift(1) takes the previous row
lnret=lnret.dropna()  #drop missing values

#Choose a set of weights for each stock (equally weighted)
ew=np.ones(N)/N

#Calculate returns for the portfolio
hist_ret_ew=(ew*lnret).sum(axis=1)

#Calculate its mean and std_dev
hist_mean=hist_ret_ew.mean()
hist_std=hist_ret_ew.std()

#Calculate the N-day return
hist_Nday_ret=hist_ret_ew.rolling(window=days).sum()

# Convert returns to dollar values for the histogram
hist_Nday_ret_dollars=hist_Nday_ret*portfolio_value

#Parametric
from scipy.stats import norm
confidence_levels=[0.9, 0.95, 0.99]
Vars_param=[]
for c in confidence_levels:
    z=norm.ppf(1-c)
    Var=(hist_mean*days - abs(z)*hist_std*np.sqrt(days))*portfolio_value
    Vars_param.append(Var)

#Historical
Vars_hist=[]
for c in confidence_levels:
    Var=np.nanpercentile(hist_Nday_ret_dollars, (1-c)*100)
    Vars_hist.append(Var)

#Monte Carlo
np.random.seed(100)
numsims=10000
simulated_values=np.zeros(numsims)
for i in range(numsims):
    z=np.random.normal(0,1) #simulation part
    deviation=z*hist_std*np.sqrt(days)
    simulated_return=(hist_mean*days)+deviation
    scenario_value=simulated_return*portfolio_value
    simulated_values[i]=scenario_value

#Take the bottom 1, 5, and 10% of the simulated values
Vars_monte=[]
for c in confidence_levels:
    Var=np.nanpercentile(simulated_values, (1-c)*100)
    Vars_monte.append(Var)

#show VaRs in df
pd.DataFrame(list(zip(Vars_param, Vars_hist, Vars_monte))
             , index=confidence_levels
            , columns=['Param','Hist', 'Monte'])

[*********************100%***********************]  29 of 29 completed


,Param,Hist,Monte
0.90,-5.999413,-4.180333,-6.008532
0.95,-8.174941,-6.853933,-8.295007
0.99,-12.255868,-13.170612,-12.181303


In [14]:
df=pd.DataFrame(pd.read_excel('CompanyIDs for Project 2024.xlsx'))
dfs=df.drop([10])
project_ciks=dfs['CIK Number']
projectids=pd.read_excel('CompanyIDs for Project 2024.xlsx').drop([10])
project_ciks=list(map(str,projectids['CIK Number']))

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}


TenKs = pd.DataFrame()


for i in range(1, 5):
    masterurl = f'https://www.sec.gov/Archives/edgar/full-index/2023/QTR{i}/master.idx' 
    content = requests.get(masterurl, headers=headers).text
    masterdata = content[content.find('CIK'):]
    datarows = masterdata.split('\n')
    filing_columns = datarows[0].split('|')
    filings = pd.DataFrame([row.split('|') for row in datarows[1:]], columns=filing_columns)
    TenKfiles = filings[filings['Form Type'] == '10-K']

    TenKs = pd.concat([TenKs, TenKfiles], ignore_index=True)
TenKs

,CIK,Company Name,Form Type,Date Filed,Filename
0,1000209,MEDALLION FINANCIAL CORP,10-K,2023-03-10,edgar/data/1000209/0000950170-23-007273.txt
1,1000228,HENRY SCHEIN INC,10-K,2023-02-21,edgar/data/1000228/0001000228-23-000011.txt
2,1000229,CORE LABORATORIES N V,10-K,2023-02-10,edgar/data/1000229/0000950170-23-002412.txt
3,1000298,IMPAC MORTGAGE HOLDINGS INC,10-K,2023-03-16,edgar/data/1000298/0001558370-23-004051.txt
4,1000623,"Mativ Holdings, Inc.",10-K,2023-03-01,edgar/data/1000623/0001000623-23-000021.txt
...,...,...,...,...,...
7361,932021,GLOBAL TECHNOLOGIES LTD,10-K,2023-12-29,edgar/data/932021/0001493152-23-046428.txt
7362,933974,"Azenta, Inc.",10-K,2023-11-21,edgar/data/933974/0001558370-23-019262.txt
7363,935419,"RCI HOSPITALITY HOLDINGS, INC.",10-K,2023-12-14,edgar/data/935419/0001628280-23-041580.txt
7364,936395,CIENA CORP,10-K,2023-12-15,edgar/data/936395/0000936395-23-000044.txt


In [15]:
len(project_ciks)

31

In [16]:
dowfilings=TenKs[TenKs['CIK'].isin(project_ciks)]
len(dowfilings)

29

In [17]:
import re
from collections import Counter
import nltk
nltk.download('punkt_tab')
import string
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('omw-1.4')
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemma = WordNetLemmatizer()

with open('LM_Positive.txt') as file:
    LM_Positive=file.read().lower()
with open('LM_Negative.txt') as file:
    LM_Negative=file.read().lower()
with open('LM_Uncertainty.txt') as file:
    LM_Uncertain=file.read().lower()

positive_bag=nltk.tokenize.word_tokenize(LM_Positive)
negative_bag=nltk.tokenize.word_tokenize(LM_Negative)
uncertain_bag=nltk.tokenize.word_tokenize(LM_Uncertain)


uncertainties=[]
tones=[]
FOGs=[]
FKs=[]
for index, row in dowfilings.iterrows():
    url="https://www.sec.gov//Archives/"+row['Filename']
    content=requests.get(url, headers=headers).text
    soup=BeautifulSoup(content, 'xml')
    text=soup.get_text().lower()
    words = nltk.tokenize.word_tokenize(text)

    nswords=[w for w in words if not(w in stopwords) and w]
    lemmawords=[lemma.lemmatize(w)for w in nswords]
    docsents=nltk.tokenize.sent_tokenize(text)
    # Calculate word counts and word frequencies
    total_words = len(lemmawords)
    total_sentences = len(docsents)

        # Calculate word counts and word frequencies
    total_words = len(lemmawords)
    total_sentences = len(docsents)

        # Calculate uncertain word percentage
    uncertain_words = [w for w in lemmawords if w in uncertain_bag]
    uncertain_count=len(uncertain_words)
    uncertain_percentage = (uncertain_count / total_words) * 100

    # Calculate word counts and word frequencies
    total_words = len(lemmawords)
    total_sentences = len(docsents)

    poswords=[w for w in lemmawords if w in positive_bag]
    negwords=[w for w in lemmawords if w in negative_bag]
    positive_count=len(poswords)
    negative_count=len(negwords)
    # Calculate LM Tone
    tone = ((positive_count - negative_count) / total_words) * 100

    nswords=[w for w in words if not(w in stopwords) and w]
    lemmawords=[lemma.lemmatize(w)for w in nswords]
    docsents=nltk.tokenize.sent_tokenize(text)
    # Calculate word counts and word frequencies
    total_words = len(lemmawords)
    total_sentences = len(docsents)

     # Calculate FK Readability
    def numsyllables(words,d):
    #receives a string word
    #receives a dictionary 'd'
        try:
            return len([y for y in d[w][0] if y[-1].isdigit()])

        except:
            return -999

    # Calculate FOG index
    from nltk.corpus import cmudict
    nltk.download('cmudict')
    d=cmudict.dict()
    complex_words = [w for w in lemmawords if numsyllables(w,d)>2]
    N_complex=len(complex_words)
    fog_index = 0.4 * ((total_words / total_sentences) + (N_complex / total_words))

    numsyllables(words, d)
    syllable_list=[numsyllables(w,d) for w in lemmawords if numsyllables(w,d)>0]
    total_syllables=np.array(syllable_list).sum()
    Read=206.835-1.015*(total_words/total_sentences)-84.6*(total_syllables/total_words)

    uncertainties.append(uncertain_percentage)
    tones.append(tone)
    FOGs.append(fog_index)
    FKs.append(Read)

results = {'Uncertainty': uncertainties,'Tone': tones,'FOG Index': FOGs,'FK Readability': FKs}
df = pd.DataFrame(results)
df

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\rebecca\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package cmudict to
[

,Uncertainty,Tone,FOG Index,FK Readability
0,1.011490,-0.916758,14.222673,170.744967
1,2.005531,-1.294259,19.181408,158.162177
2,1.128276,-0.672443,20.979276,153.600087
3,1.182126,-0.470308,19.837020,156.498562
4,1.583081,-1.499437,19.769539,156.669795
5,1.269194,-1.197308,18.248817,160.528626
6,1.330813,-0.536460,17.277551,162.993214
7,1.237441,-1.036167,18.782154,159.175284
8,1.269470,-1.306513,17.178889,163.243569
9,1.212110,-0.703278,19.688014,156.876664


In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

np.random.seed(1)

start_date=date(2020,11,2)
end_date=date(2021,1,2)
dailyprc=yf.download(dow_tickers, start_date, end_date, auto_adjust=False)

var=dailyprc['Adj Close'].var()
results = {'Uncertainty': uncertainties,'Tone': tones,'FOG Index': FOGs,'FK Readability': FKs,'Variance':var}
vars_df=pd.DataFrame(dailyprc['Adj Close'].var(), columns=['Vars'])
df = pd.DataFrame(results)
df

[*********************100%***********************]  29 of 29 completed


,Uncertainty,Tone,FOG Index,FK Readability,Variance
Ticker,,,,,
AAPL,1.011490,-0.916758,14.222673,170.744967,42.164755
AMGN,2.005531,-1.294259,19.181408,158.162177,20.077367
AMZN,1.128276,-0.672443,20.979276,153.600087,13.748585
AXP,1.182126,-0.470308,19.837020,156.498562,55.860942
BA,1.583081,-1.499437,19.769539,156.669795,632.869583
CAT,1.269194,-1.197308,18.248817,160.528626,29.511908
CRM,1.330813,-0.536460,17.277551,162.993214,226.252007
CSCO,1.237441,-1.036167,18.782154,159.175284,5.848155
CVX,1.269470,-1.306513,17.178889,163.243569,29.350086


In [19]:
""""Boeing, Disney, Goldman Sachs, and Salesforce, companies with highest variance oddly don't match tickers with the highest
uncertainty scores: Amgen, Nike, United HealthCare. The companies with highest tone scores:
IBM, Intel, and Travelers Companies Inc have moderate to low variance which seems appropriate.
Home Depot, Johnson & Johnson, MCD, and Walmart have the worst FOG index scores with low variances.
According to the FK scores all of the 10ks have similar readability levels, bwetween 145-175,
however the scores show that all of the 10ks have poor readability."""

'"Boeing, Disney, Goldman Sachs, and Salesforce, companies with highest variance oddly don\'t match tickers with the highest\nuncertainty scores: Amgen, Nike, United HealthCare. The companies with highest tone scores:\nIBM, Intel, and Travelers Companies Inc have moderate to low variance which seems appropriate.\nHome Depot, Johnson & Johnson, MCD, and Walmart have the worst FOG index scores with low variances.\nAccording to the FK scores all of the 10ks have similar readability levels, bwetween 145-175,\nhowever the scores show that all of the 10ks have poor readability.'

In [20]:
ticker='NVDA'
start_date=date(2010,11,2)
end_date=date(2023,12,31)
dailyprc=yf.download(ticker, start_date, end_date, auto_adjust=False)

dailyprc['Ch_AdjClose'] = dailyprc['Adj Close'].diff()
dailyprc['Ch_AdjClose'].dropna()

[*********************100%***********************]  1 of 1 completed


Date
2010-11-03    0.002063
2010-11-04    0.000458
2010-11-05    0.004814
2010-11-08    0.001146
2010-11-09   -0.001605
                ...   
2023-12-22   -0.159920
2023-12-26    0.448769
2023-12-27    0.137936
2023-12-28    0.104946
2023-12-29    0.000000
Name: Ch_AdjClose, Length: 3311, dtype: float64

In [21]:
ticker='KO'
start_date=date(2010,11,2)
end_date=date(2023,12,31)
dailyprc=yf.download(ticker, start_date, end_date, auto_adjust=False)
#Flatten multi index columns (remove 'KO' level, keep only 'Close', 'Volume', etc.)
if isinstance(dailyprc.columns, pd.MultiIndex):
    dailyprc.columns = dailyprc.columns.get_level_values(0)

dailyprc['Ch_AdjClose'] = dailyprc['Adj Close'].diff()
dailyprc['Ch_AdjClose'].dropna()

dailyprc['Range'] = dailyprc['High'] - dailyprc['Low']
dailyprc['Range'].dropna()

dailyprc['RangeClose'] = (dailyprc['Close'] - dailyprc['Low']) / (dailyprc['High'] - dailyprc['Low'])
dailyprc['RangeClose'].dropna()

dailyprc['OpenHigher'] = (dailyprc['Open'] > dailyprc['Close'].shift(1)).astype(int)
dailyprc['OpenHigher'].dropna()

dailyprc['Volume_7d_MA'] = dailyprc['Volume'].rolling(window=7).mean()
dailyprc['Volume_7d_MA'].dropna()
dailyprc['Volume_14d_MA'] = dailyprc['Volume'].rolling(window=14).mean()
dailyprc['Volume_14d_MA'].dropna()

dailyprc['VolumeUp'] = (dailyprc['Volume_7d_MA'] > dailyprc['Volume_14d_MA']).astype(int)

# CloseUp
dailyprc['Close_7d_MA'] = dailyprc['Close'].rolling(window=7).mean()
dailyprc['Close_14d_MA'] = dailyprc['Close'].rolling(window=14).mean()
dailyprc['CloseUp'] = (dailyprc['Close_7d_MA'] > dailyprc['Close_14d_MA']).astype(int)

dailyprc['Range_7d_MA'] = dailyprc['Range'].rolling(window=7).mean()
dailyprc['Range_14d_MA'] = dailyprc['Range'].rolling(window=14).mean()
dailyprc['RangeUp'] = (dailyprc['Range_7d_MA'] > dailyprc['Range_14d_MA']).astype(int)

dailyprc['RangeClose_7d_MA'] = dailyprc['RangeClose'].rolling(window=7).mean()
dailyprc['RangeClose_14d_MA'] = dailyprc['RangeClose'].rolling(window=14).mean()

dailyprc['OpenHigher_7d_MA'] = dailyprc['OpenHigher'].rolling(window=7).mean()

dailyprc['OpenHigher_14d_MA'] = dailyprc['OpenHigher'].rolling(window=14).mean()

dailyprc['CurrVolUp'] = (dailyprc['Volume'] > dailyprc['Volume_7d_MA']).astype(int)

dailyprc['CurrCloseUp'] = (dailyprc['Close'] > dailyprc['Close_7d_MA']).astype(int)

dailyprc['CurrRangeUp'] = (dailyprc['Range'] > dailyprc['Range_7d_MA']).astype(int)

dailyprc['L14'] = dailyprc['Low'].rolling(window=14).min()
dailyprc['H14'] = dailyprc['High'].rolling(window=14).max()

dailyprc['SO'] = (100 * (dailyprc['Close'] - dailyprc['L14']) / (dailyprc['H14'] - dailyprc['L14'])).round(2)

dailyprc['R14'] = dailyprc['H14'] - dailyprc['L14']

dailyprc['Y'] = (dailyprc['Ch_AdjClose'].shift(-1) > 0).astype(int)

X = dailyprc[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
              'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]

X = X.dropna()
y = dailyprc['Y'].loc[X.index]

holdout_df = dailyprc.loc['2023-01-01':'2023-12-31'].copy()

model_df = dailyprc.loc['2010-01-01':'2022-12-31']

X_model = model_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                    'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]
y_model = model_df['Y']

X_model = X_model.dropna()
y_model = y_model.loc[X_model.index]

X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.2, random_state=123)

[*********************100%***********************]  1 of 1 completed


In [22]:
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

LRmodel=LinearRegression()
LRmod=LRmodel.fit(X_train, y_train)

lassomodel = Lasso(alpha=0.1)
Lassomod=lassomodel.fit(X_train, y_train)

logistic_model = LogisticRegression()
Logmod=logistic_model.fit(X_train, y_train)

knn_model = KNeighborsClassifier(n_neighbors=3)
Knnmod=knn_model.fit(X_train, y_train)

dt_model = DecisionTreeClassifier(max_depth=3)
DTmod=dt_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=5)
RFmod=rf_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(max_depth=3, n_estimators=5)
GBmod=gb_model.fit(X_train, y_train)

models=[LRmodel,Lassomod,Logmod,Knnmod,DTmod,RFmod,GBmod]
mse_results = []

for m in models:
    yhat=m.predict(X_test) #predict yvalues using only the test data
    y_recession_test=(yhat>.5).astype(int) #convert True/False into 0/1
    mean_squared_error(y_test,y_recession_test)
    mse_results.append(mean_squared_error(y_test,y_recession_test))

pd.DataFrame(mse_results,index=['LRmodel','Lassomod','Logmod','Knnmod','DTmod','RFmod','GBmod'],columns=['MSEs'])

,MSEs
LRmodel,0.511475
Lassomod,0.496721
Logmod,0.516393
Knnmod,0.511475
DTmod,0.498361
RFmod,0.493443
GBmod,0.495082


In [23]:
#RF Model and GB Model had lowest MSE (best according to above df), take random forests
yhat_holdout = RFmod.predict(holdout_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                                             'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']])
# Convert predicted values to 0 or 1
y_recession_holdout = (yhat_holdout > 0.5).astype(int)

y_holdout = holdout_df['Y']

mse_holdout = mean_squared_error(y_holdout, y_recession_holdout)

mse_holdout

0.532

In [24]:
y_recession_holdout #what model predicts each day (up/down)
dailyreturn=holdout_df['Adj Close'].pct_change()#actual return of the stock on each day in 2023 (Percent change in Adj Close)

holdout_df['MyModelReturn'] = np.where(y_recession_holdout == 1, dailyreturn.shift(-1), 0)

# Calculate the total return of "MyModelReturn" for year
cumreturn = np.cumprod(1 + holdout_df['MyModelReturn'] )

#calculate passive return
passiveret=np.cumprod(1+holdout_df['Adj Close'].pct_change())
float(passiveret.iloc[-1])

0.9656744668984041

In [25]:
float(cumreturn.dropna()[-1]) #what is model return

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2860050230.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(cumreturn.dropna()[-1]) #what is model return


0.9401747673112129

In [26]:
diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]
float(diff_in_return) #compare passive and model investment strategies

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\3227223321.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]


-0.025499699587191227

In [30]:
np.random.seed(1)
ticker='VZ'
start_date=date(2010,11,2)
end_date=date(2023,12,31)
dailyprc=yf.download(ticker, start_date, end_date, auto_adjust=False)
if isinstance(dailyprc.columns, pd.MultiIndex):
    dailyprc.columns = dailyprc.columns.get_level_values(0)

dailyprc['Ch_AdjClose'] = dailyprc['Adj Close'].diff()

dailyprc['Range'] = dailyprc['High'] - dailyprc['Low']

dailyprc['RangeClose'] = (dailyprc['Close'] - dailyprc['Low']) / (dailyprc['High'] - dailyprc['Low'])

dailyprc['OpenHigher'] = (dailyprc['Open'] > dailyprc['Close'].shift(1)).astype(int)

dailyprc['Volume_7d_MA'] = dailyprc['Volume'].rolling(window=7).mean()
dailyprc['Volume_14d_MA'] = dailyprc['Volume'].rolling(window=14).mean()

dailyprc['VolumeUp'] = (dailyprc['Volume_7d_MA'] > dailyprc['Volume_14d_MA']).astype(int)

# CloseUp
dailyprc['Close_7d_MA'] = dailyprc['Close'].rolling(window=7).mean()
dailyprc['Close_14d_MA'] = dailyprc['Close'].rolling(window=14).mean()
dailyprc['CloseUp'] = (dailyprc['Close_7d_MA'] > dailyprc['Close_14d_MA']).astype(int)

dailyprc['Range_7d_MA'] = dailyprc['Range'].rolling(window=7).mean()
dailyprc['Range_14d_MA'] = dailyprc['Range'].rolling(window=14).mean()
dailyprc['RangeUp'] = (dailyprc['Range_7d_MA'] > dailyprc['Range_14d_MA']).astype(int)

dailyprc['RangeClose_7d_MA'] = dailyprc['RangeClose'].rolling(window=7).mean()
dailyprc['RangeClose_14d_MA'] = dailyprc['RangeClose'].rolling(window=14).mean()

dailyprc['OpenHigher_7d_MA'] = dailyprc['OpenHigher'].rolling(window=7).mean()
dailyprc['OpenHigher_14d_MA'] = dailyprc['OpenHigher'].rolling(window=14).mean()

dailyprc['CurrVolUp'] = (dailyprc['Volume'] > dailyprc['Volume_7d_MA']).astype(int)

dailyprc['CurrCloseUp'] = (dailyprc['Close'] > dailyprc['Close_7d_MA']).astype(int)

dailyprc['CurrRangeUp'] = (dailyprc['Range'] > dailyprc['Range_7d_MA']).astype(int)

dailyprc['L14'] = dailyprc['Low'].rolling(window=14).min()
dailyprc['H14'] = dailyprc['High'].rolling(window=14).max()

dailyprc['SO'] = (100 * (dailyprc['Close'] - dailyprc['L14']) / (dailyprc['H14'] - dailyprc['L14'])).round(2)

dailyprc['R14'] = dailyprc['H14'] - dailyprc['L14']

dailyprc.dropna(inplace=True)

dailyprc['Y'] = (dailyprc['Ch_AdjClose'].shift(-1) > 0).astype(int)

X = dailyprc[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
              'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]

X.dropna()
y = dailyprc['Y'].loc[X.index]

holdout_df = dailyprc.loc['2023-01-01':'2023-12-31'].copy()

model_df = dailyprc.loc['2010-01-01':'2022-12-31']

X_model = model_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                    'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]
y_model = model_df['Y']

X_model.dropna()

X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.2, random_state=123)

from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

LRmodel=LinearRegression()
LRmod=LRmodel.fit(X_train, y_train)

lassomodel = Lasso(alpha=0.1)
Lassomod=lassomodel.fit(X_train, y_train)

logistic_model = LogisticRegression()
Logmod=logistic_model.fit(X_train, y_train)

knn_model = KNeighborsClassifier(n_neighbors=3)
Knnmod=knn_model.fit(X_train, y_train)

dt_model = DecisionTreeClassifier(max_depth=3)
DTmod=dt_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=5)
RFmod=rf_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(max_depth=3, n_estimators=5)
GBmod=gb_model.fit(X_train, y_train)

models=[LRmodel,Lassomod,Logmod,Knnmod,DTmod,RFmod,GBmod]
mse_results = []

for m in models:
    yhat=m.predict(X_test) #predict yvalues using only the test data
    y_recession_test=(yhat>.5).astype(int) #convert True/False into 0/1
    mean_squared_error(y_test,y_recession_test)
    mse_results.append(mean_squared_error(y_test,y_recession_test))

pd.DataFrame(mse_results,index=['LRmodel','Lassomod','Logmod','Knnmod','DTmod','RFmod','GBmod'],columns=['MSEs'])

[*********************100%***********************]  1 of 1 completed


,MSEs
LRmodel,0.491803
Lassomod,0.508197
Logmod,0.491803
Knnmod,0.480328
DTmod,0.488525
RFmod,0.501639
GBmod,0.498361


In [31]:
#Knn Model has lowest MSE (is best according to above df)
yhat_holdout = Knnmod.predict(holdout_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                                             'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']])
# Convert predicted values to 0 or 1
y_recession_holdout = (yhat_holdout > 0.5).astype(int)

y_holdout = holdout_df['Y']

mse_holdout = mean_squared_error(y_holdout, y_recession_holdout)

mse_holdout

0.512

In [34]:
y_recession_holdout #what model predicts each day (up/down)
dailyreturn=holdout_df['Adj Close'].pct_change()#actual return of the stock on each day in 2023 (Percent change in Adj Close)

holdout_df['MyModelReturn'] = np.where(y_recession_holdout == 1, dailyreturn.shift(-1), 0)

# Calculate the total return of "MyModelReturn" for year
cumreturn = np.cumprod(1 + holdout_df['MyModelReturn'] )

passiveret=np.cumprod(1+holdout_df['Adj Close'].pct_change())
float(passiveret[-1])

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\2653581959.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(passiveret[-1])


1.0086990236492699

In [36]:
float(cumreturn.dropna()[-1])

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\631100883.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(cumreturn.dropna()[-1])


0.906791018578015

In [38]:
diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]
float(diff_in_return)

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\3608036455.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]


-0.10190800507125486

In [59]:
np.random.seed(1)
ticker='BA'
start_date=date(2010,11,2)
end_date=date(2023,12,31)
dailyprc=yf.download(ticker, start_date, end_date, auto_adjust=False)
if isinstance(dailyprc.columns, pd.MultiIndex):
    dailyprc.columns = dailyprc.columns.get_level_values(0)

dailyprc['Ch_AdjClose'] = dailyprc['Adj Close'].diff()

dailyprc['Range'] = dailyprc['High'] - dailyprc['Low']

dailyprc['RangeClose'] = (dailyprc['Close'] - dailyprc['Low']) / (dailyprc['High'] - dailyprc['Low'])

dailyprc['OpenHigher'] = (dailyprc['Open'] > dailyprc['Close'].shift(1)).astype(int)

dailyprc['Volume_7d_MA'] = dailyprc['Volume'].rolling(window=7).mean()
dailyprc['Volume_14d_MA'] = dailyprc['Volume'].rolling(window=14).mean()

dailyprc['VolumeUp'] = (dailyprc['Volume_7d_MA'] > dailyprc['Volume_14d_MA']).astype(int)

# CloseUp
dailyprc['Close_7d_MA'] = dailyprc['Close'].rolling(window=7).mean()
dailyprc['Close_14d_MA'] = dailyprc['Close'].rolling(window=14).mean()
dailyprc['CloseUp'] = (dailyprc['Close_7d_MA'] > dailyprc['Close_14d_MA']).astype(int)

dailyprc['Range_7d_MA'] = dailyprc['Range'].rolling(window=7).mean()
dailyprc['Range_14d_MA'] = dailyprc['Range'].rolling(window=14).mean()
dailyprc['RangeUp'] = (dailyprc['Range_7d_MA'] > dailyprc['Range_14d_MA']).astype(int)

dailyprc['RangeClose_7d_MA'] = dailyprc['RangeClose'].rolling(window=7).mean()
dailyprc['RangeClose_14d_MA'] = dailyprc['RangeClose'].rolling(window=14).mean()

dailyprc['OpenHigher_7d_MA'] = dailyprc['OpenHigher'].rolling(window=7).mean()
dailyprc['OpenHigher_14d_MA'] = dailyprc['OpenHigher'].rolling(window=14).mean()

dailyprc['CurrVolUp'] = (dailyprc['Volume'] > dailyprc['Volume_7d_MA']).astype(int)

dailyprc['CurrCloseUp'] = (dailyprc['Close'] > dailyprc['Close_7d_MA']).astype(int)

dailyprc['CurrRangeUp'] = (dailyprc['Range'] > dailyprc['Range_7d_MA']).astype(int)

dailyprc['L14'] = dailyprc['Low'].rolling(window=14).min()
dailyprc['H14'] = dailyprc['High'].rolling(window=14).max()

dailyprc['SO'] = (100 * (dailyprc['Close'] - dailyprc['L14']) / (dailyprc['H14'] - dailyprc['L14'])).round(2)

dailyprc['R14'] = dailyprc['H14'] - dailyprc['L14']

dailyprc.dropna(inplace=True)

dailyprc['Y'] = (dailyprc['Ch_AdjClose'].shift(-1) > 0).astype(int)

X = dailyprc[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
              'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]

X.dropna()
y = dailyprc['Y'].loc[X.index]

holdout_df = dailyprc.loc['2023-01-01':'2023-12-31'].copy()

model_df = dailyprc.loc['2010-01-01':'2022-12-31'].copy()

X_model = model_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                    'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']]
y_model = model_df['Y']

X_model.dropna()

X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.2, random_state=123)

from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error

LRmodel=LinearRegression()
LRmod=LRmodel.fit(X_train, y_train)

lassomodel = Lasso(alpha=0.1)
Lassomod=lassomodel.fit(X_train, y_train)

logistic_model = LogisticRegression()
Logmod=logistic_model.fit(X_train, y_train)

knn_model = KNeighborsClassifier(n_neighbors=3)
Knnmod=knn_model.fit(X_train, y_train)

dt_model = DecisionTreeClassifier(max_depth=3)
DTmod=dt_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=5)
RFmod=rf_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(max_depth=3, n_estimators=5)
GBmod=gb_model.fit(X_train, y_train)

models=[LRmodel,Lassomod,Logmod,Knnmod,DTmod,RFmod,GBmod]
mse_results = []

for m in models:
    yhat=m.predict(X_test) #predict yvalues using only the test data
    y_recession_test=(yhat>.5).astype(int) #convert True/False into 0/1
    mean_squared_error(y_test,y_recession_test)
    mse_results.append(mean_squared_error(y_test,y_recession_test))

pd.DataFrame(mse_results,index=['LRmodel','Lassomod','Logmod','Knnmod','DTmod','RFmod','GBmod'],columns=['MSEs'])

[*********************100%***********************]  1 of 1 completed


,MSEs
LRmodel,0.498361
Lassomod,0.501639
Logmod,0.496721
Knnmod,0.467213
DTmod,0.490164
RFmod,0.468852
GBmod,0.519672


In [60]:
yhat_holdout = RFmod.predict(holdout_df[['RangeClose', 'OpenHigher', 'VolumeUp', 'CloseUp', 'RangeUp', 'CurrVolUp',
                                             'CurrCloseUp', 'CurrRangeUp', 'SO', 'R14']])
# Convert predicted values to 0 or 1
y_recession_holdout = (yhat_holdout > 0.5).astype(int)

y_holdout = holdout_df['Y']

mse_holdout = mean_squared_error(y_holdout, y_recession_holdout)

mse_holdout

0.516

In [61]:
y_recession_holdout #what model predicts each day (up/down)
dailyreturn=holdout_df['Adj Close'].pct_change()#actual return of the stock on each day in 2023 (Percent change in Adj Close)

holdout_df['MyModelReturn'] = np.where(y_recession_holdout == 1, dailyreturn.shift(-1), 0)

# Calculate the total return of "MyModelReturn" for year
cumreturn = np.cumprod(1 + holdout_df['MyModelReturn'] )

passiveret=np.cumprod(1+holdout_df['Adj Close'].pct_change())
float(passiveret.iloc[-1])

1.3340498719297265

In [62]:
float(cumreturn.dropna()[-1])

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\631100883.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  float(cumreturn.dropna()[-1])


1.2191721734001633

In [63]:
diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]
float(diff_in_return)

C:\Users\rebecca\AppData\Local\Temp\ipykernel_15600\3608036455.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  diff_in_return=cumreturn.dropna()[-1]-passiveret[-1]


-0.1148776985295632